In [1]:
import math

import awkward as ak
import hist
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import uproot
import vector

print("uproot version", uproot.__version__)
print("awkward version", ak.__version__)
print("numpy version", np.__version__)
print("matplotlib version", matplotlib.__version__)
print("hist version", hist.__version__)
print("vector version", vector.__version__)
!ls -alh /pbs/throng/training/nantes-m2-rps-exp/data/*291263*

uproot version 5.5.1
awkward version 2.7.2
numpy version 2.0.2
matplotlib version 3.9.0
hist version 2.8.0
vector version 1.5.2
-rwxrwxr-x 1 2830 training 414M Aug 30  2023 /pbs/throng/training/nantes-m2-rps-exp/data/run291263.data.root
-rwxrwxr-x 1 2830 training  26M Aug 30  2023 /pbs/throng/training/nantes-m2-rps-exp/data/run291263.mc.root


In [2]:
file = uproot.open("/pbs/throng/training/nantes-m2-rps-exp/data/run291263.data.root")
file.keys()

['eventsTree;1']

In [72]:
events = file["eventsTree"]

m = events.arrays(
    ["nMuons", "Muon_Px", "Muon_Py", "Muon_Pz", "Muon_Charge", "Muon_E"],
    how="zip",
    entry_stop=10000,
)
type(m)

awkward.highlevel.Array

In [83]:
m2 = m[m.nMuons == 2]
m3 = []
for i in range(len(m2)):
    if float(m2[i].Muon.Charge[0]) * float(m2[i].Muon.Charge[1]) == -1:
        m3.append(m2[i])

for i in range(len(m3)):
    print(m3[i])
    a = m3[i].Muon.Charge[0]
    b = m3[i].Muon.Charge[1]

print(len(m3))

{nMuons: 2, Muon: [{Px: 0.576, ...}, ...]}
{nMuons: 2, Muon: [{Px: -0.694, ...}, ...]}
{nMuons: 2, Muon: [{Px: -0.901, ...}, ...]}
{nMuons: 2, Muon: [{Px: -0.205, ...}, ...]}
{nMuons: 2, Muon: [{Px: 0.513, ...}, ...]}
{nMuons: 2, Muon: [{Px: -0.39, ...}, ...]}
{nMuons: 2, Muon: [{Px: -3.8, ...}, ...]}
{nMuons: 2, Muon: [{Px: -0.648, ...}, ...]}
{nMuons: 2, Muon: [{Px: -0.284, ...}, ...]}
{nMuons: 2, Muon: [{Px: -0.446, ...}, ...]}
{nMuons: 2, Muon: [{Px: 0.424, ...}, ...]}
{nMuons: 2, Muon: [{Px: -0.258, ...}, ...]}
{nMuons: 2, Muon: [{Px: -0.273, ...}, ...]}
{nMuons: 2, Muon: [{Px: -0.0748, ...}, ...]}
{nMuons: 2, Muon: [{Px: -0.267, ...}, ...]}
{nMuons: 2, Muon: [{Px: 0.254, ...}, ...]}
{nMuons: 2, Muon: [{Px: 0.575, ...}, ...]}
{nMuons: 2, Muon: [{Px: -0.0499, ...}, ...]}
{nMuons: 2, Muon: [{Px: 0.317, ...}, ...]}
{nMuons: 2, Muon: [{Px: 0.128, ...}, ...]}
{nMuons: 2, Muon: [{Px: -0.857, ...}, ...]}
{nMuons: 2, Muon: [{Px: -0.194, ...}, ...]}
{nMuons: 2, Muon: [{Px: -0.226, ...}, ..